In [ ]:
# setting up the environment

#to interact with the Discord API and create the bot
!pip install discord 
#for numerical computations
!pip install numpy
#for saving and loading machine learning models
!pip install joblib 
#to make HTTP requests, useful for fetching data from APIs
!pip install requests 
#to use machine learning models, especially neural networks
!pip install tensorflow 
#for data manipulation and analysis
!pip install pandas
#for natural language processing tasks 
!pip install nltk 
#to allow nested event loops
!pip install nest_asyncio 
#for implementing fuzzy logic control systems
!pip install scikit-fuzzy 


In [ ]:
import discord
from discord.ext import commands
import numpy as np
import joblib
import requests
import json
import random
import pickle
from datetime import datetime
from tensorflow.keras.models import load_model
from nltk.stem import WordNetLemmatizer
import nltk
import pandas as pd
import nest_asyncio
import asyncio
import skfuzzy as fuzz
import skfuzzy.control as ctrl
import numpy as np
import pytz

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

In [ ]:
# Setting up fuzzy logic for light levels
light_level = ctrl.Antecedent(np.arange(0, 101, 1), 'light_level')
light_description = ctrl.Consequent(np.arange(0, 101, 1), 'light_description')

light_level['low'] = fuzz.trimf(light_level.universe, [0, 0, 40])
light_level['moderate'] = fuzz.trimf(light_level.universe, [20, 50, 80])
light_level['very_bright'] = fuzz.trimf(light_level.universe, [60, 100, 100])

light_description['low'] = fuzz.trimf(light_description.universe, [0, 0, 40])
light_description['moderate'] = fuzz.trimf(light_description.universe, [20, 50, 80])
light_description['very_bright'] = fuzz.trimf(light_description.universe, [60, 100, 100])

rule1 = ctrl.Rule(light_level['low'], light_description['low'])
rule2 = ctrl.Rule(light_level['moderate'], light_description['moderate'])
rule3 = ctrl.Rule(light_level['very_bright'], light_description['very_bright'])

light_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
light_sim = ctrl.ControlSystemSimulation(light_ctrl)

def get_light_description(light_level_value):
    light_sim.input['light_level'] = light_level_value
    light_sim.compute()
    light_desc_value = light_sim.output['light_description']
    if light_desc_value <= 40:
        return "Low Light"
    elif light_desc_value <= 80:
        return "Moderate Light"
    else:
        return "Very Bright"

In [ ]:
TOKEN = 'YOUR_DISCORD_BOT_TOKEN'
classifier = joblib.load('decision_tree_model.pkl')
sc = joblib.load('scaler.pkl')

with open('intents.json') as file:
    intents = json.load(file)

lemmatizer = WordNetLemmatizer()
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.keras')

from rabbit_traits import get_rabbit_traits_info
from rabbit_care import get_rabbit_care_info

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

def simulate_light_data(current_time):
    if 10 <= current_time <= 16:
        light_level_value = 90
    elif 6 <= current_time < 10 or 16 < current_time <= 18:
        light_level_value = 50
    else:
        light_level_value = 20
    light_description_value = get_light_description(light_level_value)
    return light_level_value, light_description_value

def kelvin_to_celsius(kelvin):
    return kelvin - 273.15

def get_weather_data(city='Pulau Pinang, MY'):
    weather_base_url = "https://api.openweathermap.org/data/2.5/weather?"
    api_key = 'YOUR_API_KEY'
    weather_url = weather_base_url + "appid=" + api_key + "&q=" + city

    weather_response = requests.get(weather_url).json()

    temp_celsius = kelvin_to_celsius(weather_response['main']['temp'])
    feel_like_celsius = kelvin_to_celsius(weather_response['main']['feels_like'])
    wind_speed = weather_response['wind']['speed']
    humidity = weather_response['main']['humidity']
    description = weather_response['weather'][0]['description']

    tz = pytz.timezone('Asia/Kuala_Lumpur')
    current_time = datetime.utcfromtimestamp(weather_response['dt']).replace(tzinfo=pytz.utc).astimezone(tz).hour

    light_level_value, light_description_value = simulate_light_data(current_time)

    weather_data = {
        "Temperature (°C)": temp_celsius,
        "Feels Like (°C)": feel_like_celsius,
        "Wind Speed (m/s)": wind_speed,
        "Humidity (%)": humidity,
        "LightLevel": light_level_value,
        "LightDescription": light_description_value
    }

    return weather_data

def preprocess_weather_data(weather_data):
    features = ['Humidity (%)', 'LightLevel', 'Temperature (°C)']
    df = pd.DataFrame([weather_data])
    df = df[features]
    return sc.transform(df)

def predict_health_status(city='Pulau Pinang, MY'):
    weather_data = get_weather_data(city)
    preprocessed_data = preprocess_weather_data(weather_data)
    prediction = classifier.predict(preprocessed_data)
    health_status = "Healthy" if prediction == 1 else "Not Healthy"
    return health_status, weather_data


In [ ]:
# Define intents properly
discord_intents = discord.Intents.default()
discord_intents.message_content = True

# Define the bot client
class MyClient(discord.Client):
    async def on_ready(self):
        print(f'Logged in as {self.user} (ID: {self.user.id})')
        print('------')

    async def on_message(self, message):
        if message.author.id == self.user.id:
            return

        async with message.channel.typing():
            if "temperature" in message.content.lower():
                weather_data = get_weather_data()
                response = f"The current temperature in Penang is {weather_data['Temperature (°C)']:.2f}°C."
                await message.channel.send(response)
            
            elif "humidity" in message.content.lower():
                weather_data = get_weather_data()
                response = f"The current humidity in Penang is {weather_data['Humidity (%)']}%."
                await message.channel.send(response)
            
            elif "light level" in message.content.lower():
                weather_data = get_weather_data()
                response = f"The current light level in Penang is {weather_data['LightDescription']}."
                await message.channel.send(response)

            elif "healthy" in message.content.lower() or "rabbits healthy" in message.content.lower() or "dispense food" in message.content.lower() or "feed" in message.content.lower() or "give food" in message.content.lower():
                health_status, weather_data = predict_health_status()
                if health_status == "Healthy":
                    response = (f"The rabbits are in excellent health and full of energy. It's the perfect time to feed them their favorite treats!.\n"
                                f"Current weather in Penang:\n"
                                f"Temperature: {weather_data['Temperature (°C)']:.2f}°C\n"
                                f"Feels Like: {weather_data['Feels Like (°C)']:.2f}°C\n"
                                f"Humidity: {weather_data['Humidity (%)']}%\n"
                                f"Light Level: {weather_data['LightDescription']}")
                else:
                    response = (f"The current weather conditions are affecting the rabbits' health. Please refrain from feeding them at this moment and check back later..\n"
                                f"Current weather in Penang:\n"
                                f"Temperature: {weather_data['Temperature (°C)']:.2f}°C\n"
                                f"Feels Like: {weather_data['Feels Like (°C)']:.2f}°C\n"
                                f"Humidity: {weather_data['Humidity (%)']}%\n"
                                f"Light Level: {weather_data['LightDescription']}")
                await message.channel.send(response)
                
            elif "rabbit care tips" in message.content.lower() or "take care of rabbits" in message.content.lower():
                response = get_rabbit_care_info()
                await message.channel.send(response)

            elif "rabbit traits" in message.content.lower() or "rabbit personalities" in message.content.lower() or "rabbit characteristics" in message.content.lower():
                response = get_rabbit_traits_info()
                await message.channel.send(response)

            else:
                ints = predict_class(message.content)
                res = get_response(ints, intents)
                await message.channel.send(res)

def main():
    client = MyClient(intents=discord_intents)
    client.run(TOKEN)

if __name__ == '__main__':
    main()